In [1]:
pip install paddlepaddle-gpu==2.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.0/394.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 31.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install paddlenlp==2.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=5a21b6d80926485b64f4489b3a7bc0b4211a123b3e5c2262c2bdf3a6e13884bb
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c

In [3]:
import os
import paddle
import paddlenlp
import pandas as pd
from paddlenlp.datasets import MapDataset
import functools
import numpy as np
from paddle.io import DataLoader, BatchSampler
import time
import paddle.nn.functional as F
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer


In [4]:
# 构建验证集evaluate函数
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        
    accu = metric.accumulate()
    print("eval loss: %.5f, accuracy: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()
    return accu

In [5]:
labelmap={"none":0,"happiness":1,"sadness":2,"anger":3,"surprise":4,"fear":5,"disgust":6}
traindata = pd.read_csv('/kaggle/input/datav4/train.csv', sep=None, header=0, encoding='utf-8', engine='python')
train_ds=MapDataset([{'text':d.text,'label':labelmap[d.label],'qid':str(d.qid)} for d in traindata.itertuples()])
devdata = pd.read_csv('/kaggle/input/datav4/test.csv', sep=None, header=0, encoding='utf-8', engine='python')
dev_ds=MapDataset([{'text':d.text,'label':labelmap[d.label],'qid':str(d.qid)} for d in devdata.itertuples()])
testdata = pd.read_csv('/kaggle/input/datav4/dev.csv', sep=None, header=0, encoding='utf-8', engine='python')
test_ds=MapDataset([{'text':d.text,'label':"",'qid':str(d.qid)} for d in testdata.itertuples()])
# 数据集返回为MapDataset类型
print("数据类型:", type(train_ds))
# label代表标签，qid代表数据编号，测试集中不包含标签信息
print("训练集样例:", train_ds[0])
print("验证集样例:", dev_ds[0])
print("测试集样例:", test_ds[0])

数据类型: <class 'paddlenlp.datasets.dataset.MapDataset'>
训练集样例: {'text': '回覆嗯~ 很幸福！以后我也要这样……今晚和一个本地人去了一个cafe ，到了之后他和我说这儿的老板是一对GAY ，然后我跑去看这一对，哎，好幸福啊！一起幸福的经营者一家超有feel 的cafe ，羡慕死了！今晚很开心！', 'label': 1, 'qid': '1'}
验证集样例: {'text': '书读多了未必都是好事呀，很多人都教条主义了，变成书呆子一个啊。。。。笑死我了！好一对新生代对联！愿读服输争当精品，是珍珠，必会发光。好一对新生代对联！博士生，研究生，本科生，生生不息！下联：上一届，这一届，下一届，届届失业！横批：愿读服输via 财经在线', 'label': 1, 'qid': '1'}
测试集样例: {'text': '我在看：【保姆纵火案4年后那个痛失妻儿的男人怎么样了】#跟新浪看热点#O保姆纵火案4年后那个痛失妻儿的男人怎么样了', 'label': '', 'qid': '1'}


In [6]:
model_name = "ernie-3.0-xbase-zh"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=7)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2022-11-23 16:55:24,816] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-xbase-zh'.
[2022-11-23 16:55:24,818] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_xbase_zh.pdparams and saved to /root/.paddlenlp/models/ernie-3.0-xbase-zh
[2022-11-23 16:55:24,820] [    INFO] - Downloading ernie_3.0_xbase_zh.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_xbase_zh.pdparams
100%|██████████| 1.11G/1.11G [02:15<00:00, 8.76MB/s]
W1123 16:57:42.105593    24 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 6.0, Driver API Version: 11.4, Runtime API Version: 10.2
W1123 16:57:42.152628    24 gpu_resources.cc:91] device: 0, cuDNN Version: 8.0.
[2022-11-23 16:57:50,267] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-3.0-xbase-zh'.
[2022-11-23 16:57:50,269] [    INFO

In [7]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# 数据预处理函数，利用分词器将文本转化为整数序列
def preprocess_function(examples, tokenizer, max_seq_length, is_test=False):

    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    if not is_test:
        result["labels"] = examples["label"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128)
train_ds = train_ds.map(trans_func)
dev_ds = dev_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorWithPadding(tokenizer)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
train_batch_sampler = BatchSampler(train_ds, batch_size=16, shuffle=True)
dev_batch_sampler = BatchSampler(dev_ds, batch_size=16, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
dev_data_loader = DataLoader(dataset=dev_ds, batch_sampler=dev_batch_sampler, collate_fn=collate_fn)

In [8]:
# Adam优化器、交叉熵损失函数、accuracy评价指标
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())
criterion = paddle.nn.loss.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

In [9]:
# 开始训练
import time
import paddle.nn.functional as F
epochs = 5 # 训练轮次
ckpt_dir = "ernie_ckpt" #训练过程中保存模型参数的文件夹
best_acc = 0
best_step = 0
bestmodel=ckpt_dir
global_step = 0 #迭代次数
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        # 每迭代100次，打印损失函数值、准确率、计算速度
        global_step += 1
        if global_step % 200 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,
                    200 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        # 每迭代1500次，评估当前训练的模型、保存当前模型参数和分词器的词表等
        if global_step % 2000 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            print(global_step, end=' ')
            acc_eval = evaluate(model, criterion, metric, dev_data_loader)
            if acc_eval > best_acc:
                best_acc = acc_eval
                best_step = global_step
                t=time.time()
                bestmodel=save_dir+str(t)
                model.save_pretrained(save_dir+str(t))
                tokenizer.save_pretrained(save_dir+str(t))

global step 200, epoch: 1, batch: 200, loss: 0.58825, accu: 0.60000, speed: 1.86 step/s
global step 400, epoch: 1, batch: 400, loss: 1.03246, accu: 0.65734, speed: 1.90 step/s
global step 600, epoch: 1, batch: 600, loss: 0.73723, accu: 0.69073, speed: 1.92 step/s
global step 800, epoch: 1, batch: 800, loss: 0.59192, accu: 0.69875, speed: 1.89 step/s
global step 1000, epoch: 1, batch: 1000, loss: 0.71657, accu: 0.71256, speed: 1.89 step/s
global step 1200, epoch: 1, batch: 1200, loss: 0.54276, accu: 0.72141, speed: 1.89 step/s
global step 1400, epoch: 1, batch: 1400, loss: 0.81225, accu: 0.72817, speed: 1.90 step/s
global step 1600, epoch: 1, batch: 1600, loss: 0.85994, accu: 0.73426, speed: 1.89 step/s
global step 1800, epoch: 1, batch: 1800, loss: 0.35955, accu: 0.73958, speed: 1.90 step/s
global step 2000, epoch: 1, batch: 2000, loss: 0.35070, accu: 0.74506, speed: 1.88 step/s
2000 eval loss: 0.65221, accuracy: 0.77799


[2022-11-23 17:22:22,245] [    INFO] - tokenizer config file saved in ernie_ckpt1669224139.6267614/tokenizer_config.json
[2022-11-23 17:22:22,251] [    INFO] - Special tokens file saved in ernie_ckpt1669224139.6267614/special_tokens_map.json


global step 2200, epoch: 1, batch: 2200, loss: 0.62946, accu: 0.78531, speed: 0.39 step/s
global step 2400, epoch: 1, batch: 2400, loss: 0.61989, accu: 0.79688, speed: 1.89 step/s
global step 2600, epoch: 1, batch: 2600, loss: 0.20337, accu: 0.79250, speed: 1.91 step/s
global step 2800, epoch: 1, batch: 2800, loss: 0.57439, accu: 0.79391, speed: 1.87 step/s
global step 3000, epoch: 1, batch: 3000, loss: 0.72443, accu: 0.79200, speed: 1.89 step/s
global step 3200, epoch: 1, batch: 3200, loss: 0.49517, accu: 0.79141, speed: 1.90 step/s
global step 3400, epoch: 1, batch: 3400, loss: 0.33227, accu: 0.79250, speed: 1.88 step/s
global step 3600, epoch: 1, batch: 3600, loss: 0.36389, accu: 0.79297, speed: 1.89 step/s
global step 3800, epoch: 1, batch: 3800, loss: 0.73057, accu: 0.79260, speed: 1.89 step/s
global step 4000, epoch: 1, batch: 4000, loss: 0.68889, accu: 0.79375, speed: 1.89 step/s
4000 eval loss: 0.57994, accuracy: 0.80903


[2022-11-23 17:46:48,726] [    INFO] - tokenizer config file saved in ernie_ckpt1669225605.887588/tokenizer_config.json
[2022-11-23 17:46:48,731] [    INFO] - Special tokens file saved in ernie_ckpt1669225605.887588/special_tokens_map.json


global step 4200, epoch: 1, batch: 4200, loss: 0.42890, accu: 0.80375, speed: 0.39 step/s
global step 4400, epoch: 1, batch: 4400, loss: 0.21524, accu: 0.79484, speed: 1.89 step/s
global step 4600, epoch: 1, batch: 4600, loss: 0.47163, accu: 0.79656, speed: 1.91 step/s
global step 4800, epoch: 1, batch: 4800, loss: 0.16440, accu: 0.79898, speed: 1.90 step/s
global step 5000, epoch: 1, batch: 5000, loss: 0.90844, accu: 0.80044, speed: 1.89 step/s
global step 5200, epoch: 1, batch: 5200, loss: 0.85715, accu: 0.80010, speed: 1.90 step/s
global step 5400, epoch: 1, batch: 5400, loss: 0.27794, accu: 0.80107, speed: 1.90 step/s
global step 5600, epoch: 1, batch: 5600, loss: 0.11480, accu: 0.79945, speed: 1.89 step/s
global step 5800, epoch: 1, batch: 5800, loss: 0.54374, accu: 0.79833, speed: 1.90 step/s
global step 6000, epoch: 1, batch: 6000, loss: 0.70385, accu: 0.79809, speed: 1.90 step/s
6000 eval loss: 0.54025, accuracy: 0.81771


...

global step 8200, epoch: 1, batch: 8200, loss: 0.95629, accu: 0.80625, speed: 0.39 step/s
global step 8400, epoch: 1, batch: 8400, loss: 0.55419, accu: 0.81172, speed: 1.89 step/s
global step 8600, epoch: 1, batch: 8600, loss: 0.68794, accu: 0.80990, speed: 1.92 step/s
global step 8800, epoch: 2, batch: 191, loss: 0.47074, accu: 0.81897, speed: 1.90 step/s
global step 9000, epoch: 2, batch: 391, loss: 0.39585, accu: 0.82262, speed: 1.90 step/s
global step 9200, epoch: 2, batch: 591, loss: 0.26934, accu: 0.82739, speed: 1.91 step/s
global step 9400, epoch: 2, batch: 791, loss: 0.38895, accu: 0.83018, speed: 1.91 step/s
global step 9600, epoch: 2, batch: 991, loss: 0.22957, accu: 0.83086, speed: 1.90 step/s
global step 9800, epoch: 2, batch: 1191, loss: 0.47120, accu: 0.83191, speed: 1.89 step/s
global step 10000, epoch: 2, batch: 1391, loss: 0.32068, accu: 0.83325, speed: 1.90 step/s
10000 eval loss: 0.56036, accuracy: 0.81372
global step 10200, epoch: 2, batch: 1591, loss: 0.41910, acc

[2022-11-23 19:24:07,434] [    INFO] - tokenizer config file saved in ernie_ckpt1669231444.754276/tokenizer_config.json
[2022-11-23 19:24:07,436] [    INFO] - Special tokens file saved in ernie_ckpt1669231444.754276/special_tokens_map.json


global step 12200, epoch: 2, batch: 3591, loss: 0.53443, accu: 0.84937, speed: 0.39 step/s
global step 12400, epoch: 2, batch: 3791, loss: 0.38156, accu: 0.84531, speed: 1.91 step/s
global step 12600, epoch: 2, batch: 3991, loss: 0.55776, accu: 0.84521, speed: 1.93 step/s
global step 12800, epoch: 2, batch: 4191, loss: 0.49583, accu: 0.84578, speed: 1.90 step/s
global step 13000, epoch: 2, batch: 4391, loss: 0.82489, accu: 0.84444, speed: 1.90 step/s
global step 13200, epoch: 2, batch: 4591, loss: 0.08823, accu: 0.84354, speed: 1.90 step/s
global step 13400, epoch: 2, batch: 4791, loss: 0.15964, accu: 0.84254, speed: 1.91 step/s
global step 13600, epoch: 2, batch: 4991, loss: 0.24635, accu: 0.84230, speed: 1.90 step/s
global step 13800, epoch: 2, batch: 5191, loss: 0.72128, accu: 0.84208, speed: 1.89 step/s
global step 14000, epoch: 2, batch: 5391, loss: 0.71905, accu: 0.84166, speed: 1.90 step/s
14000 eval loss: 0.53134, accuracy: 0.81900
global step 14200, epoch: 2, batch: 5591, loss

[2022-11-23 20:12:43,063] [    INFO] - tokenizer config file saved in ernie_ckpt1669234360.372969/tokenizer_config.json
[2022-11-23 20:12:43,068] [    INFO] - Special tokens file saved in ernie_ckpt1669234360.372969/special_tokens_map.json


global step 16200, epoch: 2, batch: 7591, loss: 0.25442, accu: 0.83688, speed: 0.39 step/s
global step 16400, epoch: 2, batch: 7791, loss: 0.47601, accu: 0.83594, speed: 1.90 step/s
global step 16600, epoch: 2, batch: 7991, loss: 0.46992, accu: 0.83521, speed: 1.89 step/s
global step 16800, epoch: 2, batch: 8191, loss: 0.52331, accu: 0.83648, speed: 1.89 step/s
global step 17000, epoch: 2, batch: 8391, loss: 0.53172, accu: 0.83913, speed: 1.90 step/s
global step 17200, epoch: 2, batch: 8591, loss: 0.47768, accu: 0.83958, speed: 1.91 step/s
global step 17400, epoch: 3, batch: 182, loss: 0.22967, accu: 0.84554, speed: 1.92 step/s
global step 17600, epoch: 3, batch: 382, loss: 0.75563, accu: 0.84942, speed: 1.91 step/s
global step 17800, epoch: 3, batch: 582, loss: 0.37811, accu: 0.85306, speed: 1.91 step/s
global step 18000, epoch: 3, batch: 782, loss: 0.28553, accu: 0.85529, speed: 1.91 step/s
18000 eval loss: 0.57166, accuracy: 0.82462
global step 18200, epoch: 3, batch: 982, loss: 0.4

In [10]:
# 加载ERNIR 3.0最佳模型参数
params_path = './'+bestmodel+'/model_state.pdparams'
state_dict = paddle.load(params_path)
model.set_dict(state_dict)
print('在dev集表现', end=' ')
eval_acc = evaluate(model, criterion, metric, dev_data_loader)


在dev集表现 eval loss: 0.52338, accuracy: 0.82911


In [11]:
# 测试集数据预处理，利用分词器将文本转化为整数序列
trans_func_test = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128, is_test=True)
test_ds_trans = test_ds.map(trans_func_test)

# 进行采样组batch
collate_fn_test = DataCollatorWithPadding(tokenizer)
test_batch_sampler = BatchSampler(test_ds_trans, batch_size=16, shuffle=False)
test_data_loader = DataLoader(dataset=test_ds_trans, batch_sampler=test_batch_sampler, collate_fn=collate_fn_test)

# 模型预测分类结果
label_map={0:"none",1:"happiness",2:"sadness",3:"anger",4:"surprise",5:"fear",6:"disgust"}
results = []
probs_for_8=[]
model.eval()
k=0
for batch in test_data_loader:
    if k%200==0:
        print(k)  
    input_ids, token_type_ids = batch['input_ids'], batch['token_type_ids']
    logits = model(batch['input_ids'], batch['token_type_ids'])
    probs = F.softmax(logits, axis=-1)
    #存储标签
    idx = paddle.argmax(probs, axis=1).numpy()
    idx = idx.tolist()
    preds = [label_map[i] for i in idx]
    results.extend(preds)
    #存储可能性
    probs=probs.numpy()
    probs=probs.tolist()
    probs_for_8.extend([i for i in probs])
    k+=1

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600


In [12]:
# 存储预测结果  
test_ds=MapDataset([{'text':d.text,'label':"",'qid':str(d.qid)} for d in testdata.itertuples()])

res_dir = "./results"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
with open(os.path.join(res_dir, "weibosenti"+str(time.time())+".csv"), 'w', encoding="utf8") as f:
    f.write("qid,text,prediction,none,happiness,sadness,anger,surprise,fear,disgust\n")
    for i, pred in enumerate(results):
    f.write(test_ds[i]['qid']+","+test_ds[i]['text']+","+pred+","+\
        str(probs_for_8[i][0])+","+str(probs_for_8[i][1])+","+str(probs_for_8[i][2])+","+str(probs_for_8[i][3])+","+\
        str(probs_for_8[i][4])+","+str(probs_for_8[i][5])+","+str(probs_for_8[i][6])+"\n")

IndentationError: expected an indented block (468556500.py, line 10)